In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import contextily
import matplotlib.pyplot as plt
from pointpats import centrography
from utils.data_utils import fetch_data
# The imports below expects that site_list.py exists in the src/visualisation/utils folder
# Contents of the file can be asked from the authors of this project.
from utils.sites_and_limits import sites_pattern_plotting, sites_y_x_limits

In [ ]:
def fetch_data():
  df_general = pd.read_csv(
    '../../resources/generated_data/joined_dataset.csv', index_col='NestID'
  )
  df_location = pd.read_csv( 
    '../../resources/original_data/FinlandNestDatafile.csv', index_col='NestID'
  )
  df_location21 = pd.read_csv( 
    '../../resources/original_data/Finland_nestdata2021_mod.csv', index_col='NestID'
  )
  df_location21['Year'] = 2021
  df_location = pd.concat([df_location, df_location21])
  df_location = df_location.drop(columns=['Rebuild_original','New_rebuild', 'Cuckoo_perch',
    'Date_nest_found'])
  df_general = df_general.drop(columns=['Rebuild_original', 'Cuckoo_perch',
    'Date_nest_found','Days_from_LD','Date_trial','Laydate_first_egg'])
  R = 6378137.0
  x_all = np.radians(df_location['long'])*R
  y_all = np.log(np.tan(np.pi/4+np.radians(df_location['lat']/2)))*R
  x_withD = np.radians(df_general['long'])*R
  y_withD = np.log(np.tan(np.pi/4+np.radians(df_general['lat']/2)))*R
  df_location['x']=x_all
  df_location['y']=y_all
  df_general['x']=x_withD
  df_general['y']=y_withD
  return df_general, df_location


In [ ]:
def plot_jointplot(df_location):
  joint_axes = sns.jointplot(x='long', y='lat', data=df_location, s=2.5, hue='Year', height=9,
    palette={2019: '#003456',
    2020: '#003458',
    2021: '#002212'
  })
  contextily.add_basemap(
      joint_axes.ax_joint,
      crs='EPSG:4326',
      source=contextily.providers.CartoDB.PositronNoLabels
  )
  plt.show()

In [ ]:
def plot_kdeplot(df_location):
  f, ax = plt.subplots(1, figsize=(20,19))
  sns.kdeplot(
    x=df_location['x'],
    y=df_location['y'],
    n_levels=50,
    shade=True,
    alpha=0.55,
    cmap='viridis_r'
    )

  contextily.add_basemap(
    ax,
    source=contextily.providers.CartoDB.PositronNoLabels
  )
  plt.show()


In [ ]:

df_general, df_location = fetch_data()
plot_kdeplot(df_location)

In [ ]:
plot_jointplot(df_location)

In [ ]:
# One can give variables y_lim and x_lim (both are tuples) in order to get the 
# plots same sized plots for the specific site
# The minimun and maximum from the each Site can be fetched with the following code:

def create_map_plot(df_location: pd.DataFrame, df_general: pd.DataFrame,
    year: int, y_lim = None, x_lim = None):
  mean_center_all = centrography.mean_center(df_location[['x','y']])
  #med_center_all = centrography.eucledian_median(df_location[['x','y']])
  sns.set(font_scale=2)
  joint_axes = sns.scatterplot(
    x='x', y='y', data=df_location, color='black', s=20,
  )

  joint_axes.set_title(year, fontsize=20)
  if y_lim and x_lim:
    joint_axes.set(ylim=y_lim)
    joint_axes.set(xlim=x_lim)
  if not df_general.empty:
    if not df_general[df_general.Propensity==1].empty:
      joint_axes.axes.scatter(
        x='x', y='y', data=df_general[df_general.Propensity==1],
        color='red', marker='o', s=20, label='Aggressive'
      )
      mean_center_agr = centrography.mean_center( df_general[df_general.Propensity==1][['x','y']])
      #med_center_agr = centrography.eucledian_median(df_location[df_general.Propensity==1][['x','y']])
      joint_axes.axes.scatter(
        *mean_center_agr, color='orangered', marker='x', s=50, label='Mean Agr'
      )
    
    if not df_general[df_general.Propensity==0].empty: 
      joint_axes.axes.scatter(
        x='x', y='y', data=df_general[df_general.Propensity==0],
        color='blue', marker='o', s=20, label='Shy'
      )
      mean_center_shy = centrography.mean_center( df_general[df_general.Propensity==0][['x','y']])
      #med_center_shy = centrography.eucledian_median(df_location[df_general.Propensity==0][['x','y']])
      joint_axes.axes.scatter(
        *mean_center_shy, color='indigo', marker='x', s=50, label='Mean Shy'
      )

  joint_axes.axes.scatter(
    *mean_center_all, color='magenta', marker='x', s=40, label='Mean All'
  )
  joint_axes.axes.legend()
  contextily.add_basemap(
    joint_axes.axes,
    source=contextily.providers.CartoDB.Positron
  )
  joint_axes.axes.set_axis_off()

In [ ]:
def create_plot(sites: list, df_location_year: pd.DataFrame, df_general_year: pd.DataFrame,
    year: int, y_lim: tuple, x_lim: tuple):
  df_location_tmp = df_location_year[df_location_year.Site.str.strip().isin(sites)]
  df_general_tmp = df_general_year[df_general_year.Site.str.strip().isin(sites)]
  create_map_plot(df_location_tmp, df_general_tmp, year, y_lim, x_lim)

In [ ]:
# Separate data into yearly dataframes
df_location19 = df_location[df_location['Year']==2019]
df_location20 = df_location[df_location['Year']==2020]
df_location21 = df_location[df_location['Year']==2021]

df_general19 = df_general[df_general['Year']==2019]
df_general20 = df_general[df_general['Year']==2020] 
df_general21 = df_general[df_general['Year']==2021] 

In [ ]:
sites = sites_pattern_plotting()
limits = sites_y_x_limits()

In [ ]:
ylim = limits[0][0]
xlim = limits[0][1]
plt.figure(figsize = (20,15))
plt.subplot(2,2,1)
plt.suptitle(f'{sites[0][0]} 2019-2021', fontsize=30)
create_plot(sites[0], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[0], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot(sites[0], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[1][0]
xlim = limits[1][1]
plt.figure(figsize = (20,12))
plt.subplot(1,3,1)
plt.suptitle(f'{sites[1][0]} 2019-2021', fontsize=30)
create_plot(sites[1], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(1,3,2)
create_plot(sites[1], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,3,3)
create_plot(sites[1], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[2][0]
xlim = limits[2][1]
plt.figure(figsize = (20,15))
plt.subplot(2,2,1)
plt.suptitle(f'{sites[2][0]} 2019-2021', fontsize=30)
create_plot(sites[2], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[2], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot(sites[2], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[3][0]
xlim = limits[3][1]

plt.figure(figsize = (20,10))
plt.subplot(1,3,1)
plt.suptitle(f'{sites[3][0]} 2019-2021', fontsize= 30)
create_plot(sites[3], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(1,3,2)
create_plot(sites[3], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,3,3)
create_plot(sites[3], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[4][0]
xlim = limits[4][1]
title = " and ".join(sites[4])
plt.figure(figsize = (20,10))
plt.subplot(1,3,1)
plt.suptitle(f'{title} 2019-2021', fontsize=20)
create_plot(sites[4], df_location19, df_general19,2019, ylim, xlim)
plt.subplot(1,3,2)
create_plot(sites[4], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,3,3)
create_plot(sites[4], df_location21, df_general21, 2021, ylim, xlim)
plt.show()

In [ ]:
ylim = limits[5][0]
xlim = limits[5][1]
title = " and ".join(sites[5])
plt.figure(figsize = (20,15))
plt.subplot(2,2,1)
plt.suptitle(f'{title} 2019-2021', fontsize = 30)
create_plot([sites[5][0]], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[5], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot(sites[5], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[6][0]
xlim = limits[6][1]
title = " and ".join(sites[6])
plt.figure(figsize = (20,8))
plt.subplot(2,2,1)
plt.suptitle(f'{title} 2019-2021', fontsize=30)
create_plot(sites[6], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[6], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot([sites[6][0]], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
# Vuosaari and Vikkulla
ylim = limits[7][0]
xlim = limits[7][1]
title = " and ".join(sites[7])
plt.figure(figsize = (15,15))
plt.subplot(2,2,1)
plt.suptitle(f'{title} 2019-2021', fontsize = 30)
create_plot([sites[7][0]], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[7], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot(sites[7], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
# Norra Fladet
ylim = limits[8][0]
xlim = limits[8][1]

plt.figure(figsize = (15,10))
plt.subplot(2,2,1)
plt.suptitle(f'{sites[8]} 2019-2020', fontsize=30)
create_plot(sites[8], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[8], df_location20, df_general20, 2020, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[9][0]
xlim = limits[9][1]
title = " and ".join(sites[9])

plt.figure(figsize = (20,10))
plt.subplot(2,2,1)
plt.suptitle(f'{title}  2019-2021', fontsize=30)
create_plot(sites[9], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[9], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(2,2,3)
create_plot(sites[9], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
ylim = limits[10][0]
xlim = limits[10][1]
title = " and ".join(sites[10])

plt.figure(figsize = (20,10))
plt.subplot(1,3,1)
plt.suptitle(f'{title} 2019-2021', fontsize=30)
create_plot(sites[10], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(1,3,2)
create_plot(sites[10], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,3,3)
create_plot([sites[10][1]], df_location21, df_general21,2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
ylim = limits[11][0]
xlim = limits[11][1]

plt.figure(figsize = (15,10))
plt.subplot(1,2,1)
plt.suptitle(f'{sites[11]} 2020-2021', fontsize=30)
create_plot(sites[11], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[11], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[12][0]
xlim = limits[12][1]

plt.figure(figsize = (15,3))
plt.subplot(1,2,1)
plt.suptitle(f'{sites[12]} 2020-2021', fontsize=30)
create_plot(sites[12], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[12], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[13][0]
xlim = limits[13][1]

plt.figure(figsize = (15,10))
plt.subplot(1,2,1)
plt.suptitle(f'{sites[13]} 2020-2021', fontsize=30)
create_plot(sites[13], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[13], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[14][0]
xlim = limits[14][1]

plt.figure(figsize = (15,10))
plt.subplot(1,2,1)
plt.suptitle(f'{sites[14]} 2020-2021', fontsize=30)
create_plot(sites[14], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[14], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
ylim = limits[15][0]
xlim = limits[15][1]

plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
plt.suptitle(f'{sites[15][0]} 2020-2021', fontsize=30)
create_plot(sites[15], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[15], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
ylim = limits[16][0]
xlim = limits[16][1]

title = " and ".join(sites[16])
plt.figure(figsize = (14,8))
plt.subplot(1,2,1)
plt.suptitle(f'{title} 2020-2021', fontsize=30)
create_plot(sites[16], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,2,2)
create_plot(sites[16], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout()
plt.show()

In [ ]:
ylim = limits[17][0]
xlim = limits[17][1]

plt.figure(figsize = (15,6))
plt.subplot(1,3,1)
plt.suptitle(f'{sites[17][0]} 2019-2021', fontsize= 30)
create_plot(sites[17], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(1,3,2)
create_plot(sites[17], df_location20, df_general20, 2020, ylim, xlim)
plt.subplot(1,3,3)
create_plot(sites[17], df_location21, df_general21, 2021, ylim, xlim)
plt.tight_layout() 
plt.show()

In [ ]:
# Fiskarviken
ylim = limits[18][0]
xlim = limits[18][1]

plt.figure(figsize = (8,10))
plt.subplot(2,2,1)
plt.suptitle(f'{sites[18][0]} 2019-2020', fontsize=30)
create_plot(sites[18], df_location19, df_general19, 2019, ylim, xlim)
plt.subplot(2,2,2)
create_plot(sites[18], df_location20, df_general20, 2020, ylim, xlim)
plt.tight_layout() 
plt.show()